In [8]:
# %%
import pandas as pd

class InferCSV():
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
        self.infer_df = self.infer_and_convert_data_types()
        

    def count_type(self, x):

        infer_type = 'string'

        try:
            x = int(x)
            infer_type = 'int'
        except ValueError:
            pass

        try:
            # condition for datetime format
            if len(x) >= 7 and x.count("/") == 2:
                x = pd.to_datetime(x)
                infer_type = 'date-time'

        except (ValueError, TypeError):
            pass
        return infer_type
    


    def infer_and_convert_data_types(self):
        
        df = self.df
        
        for col in df.columns:
            types = df[col].apply(self.count_type)
            types.name = "types"
            majority_type = types.value_counts(ascending=False).idxmax(axis=1)

            # remove the incorrect datetime format
            df = pd.concat([df, types], axis = 1)
            df = df.drop(df[(df['types'] != majority_type)].index)
            df = df.drop('types', axis=1)

            if majority_type == 'int':
                df[col] = df[col].astype('int8')

            elif majority_type == 'date-time':
                
                # remove incorrect datetime format
                df[col] = pd.to_datetime(df[col])

            # if majority_type == string
            else:
                if len(df[col].unique()) / len(df[col]) < 0.5:  # Example threshold for categorization
                    df[col] = pd.Categorical(df[col])

        return df

    def get_infer_types(self):
        return self.infer_df.dtypes.apply(lambda x: x.name).to_dict()


csv_file = 'sample_data.csv'

CSV = InferCSV(csv_file)
types = CSV.get_infer_types()


print("\nData types after inference:")
print(types)


# %%
df.style


Data types after inference:
{'Name': 'object', 'Birthdate': 'datetime64[ns]', 'Score': 'int8', 'Grade': 'object'}


,Name,Birthdate,Score,Grade
0,Alice,1990-01-01 00:00:00,90,A
1,Bob,1991-02-02 00:00:00,75,B
2,Charlie,1992-03-03 00:00:00,85,A
3,David,1993-04-04 00:00:00,70,B
